In [1]:
#libraries 
import numpy as np
import pandas as pd
import re
import os


#preprocessing
import string
from time import time  
from nltk.corpus import stopwords #remove stop words
chachedWords = stopwords.words('english')

#cosine similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import math
from collections import Counter


Base: https://medium.com/@adriensieg/text-similarities-da019229c894

REF: 
    
https://arxiv.org/pdf/1301.3781.pdf
    
https://www.aclweb.org/anthology/N19-1181.pdf
    
http://proceedings.mlr.press/v37/kusnerb15.pdf

https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/37842.pdf

https://arxiv.org/pdf/1609.08144.pdf

In [2]:
#df = pd.read_csv(r"C:\Users\hirom\OneDrive - NOVAIMS\NOVA IMS\OneDrive_1_25-02-2021\Text Mining\Project\corpus\de-en\scores.csv")
df = pd.read_csv(r"C:\Users\hirom\Documents\GitHub\textmining_project\test\cs-en\scores.csv")
df

,source,reference,translation
0,"Památník, důstojné pietní místo, stojí vůlí dě...","The monument, a dignified piecemeal place, sta...","The memorial, a solemn place of commemoration,..."
1,Pracovník centra Čang Č-čung sdělil agentuře N...,Centre worker Zhang Zu-chung told the New Chin...,Centre worker Chang Chi-Chung told New China t...
2,Veterináři nicméně odeberou namátkové vzorky v...,"However, veterinarians take random samples of ...","However, veterinarians are taking samples of e..."
3,Uživatel @TheePharoah jí neustále retweetoval ...,User @ TheePharoah constantly retweeted her po...,A user with the handle @TheePharoah was being ...
4,Lucii bylo tehdy pouhých 19 let a rozhodně net...,Lucia was only 19 at the time and certainly ha...,"At that time, Lucie was only 19 years old, and..."
...,...,...,...
8727,Convercent vybral v rámci své únorové kampaně ...,Convercent picked in the framework of its Febr...,Convercent raised $10 million in funding in Fe...
8728,Kde všude mohou Češi volit?,Where can the Czechs to vote?,Where in the world can Czechs vote?
8729,Hlavní zkoušky letošní rekonstrukce započaly v...,The main test of this year's renovation began ...,Rehearsals for this year's production began in...
8730,"Většina z tisíců vražd, ke kterým dochází každ...",Most of the thousands of murders occurring eve...,As with most of the thousands of murders tied ...


## Preprocessing 

In [3]:
def preprocessing2(sentence):
    return " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in sentence.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in chachedWords)
     

In [4]:
df['p_reference'] = df.apply(lambda x:preprocessing2(x['reference']),axis=1)

In [5]:
df['p_translation'] = df.apply(lambda x:preprocessing2(x['translation']),axis=1)

In [6]:
def delete_sw(sentence):
    stop_words = stopwords.words('english')
    processed = sentence.split()
    clean = [w for w in processed if w not in chachedWords]
    return clean 

In [7]:
df_cp = df.copy()

In [8]:
df_cp['w_reference'] = df_cp.apply(lambda x:delete_sw(x['p_reference']),axis=1)

In [9]:
df_cp['w_translation'] = df_cp.apply(lambda x:delete_sw(x['p_translation']),axis=1)

## Embedding

https://www.analyticsvidhya.com/blog/2020/08/top-4-sentence-embedding-techniques-using-python/

Doc2Vec

In [10]:
# import
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(df_cp['w_translation'])]
tagged_data

C:\Users\hirom\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


[TaggedDocument(words=['memorial', 'solemn', 'place', 'commemoration', 'stands', 'thanks', 'history', 'almost', 'immediate', 'vicinity', 'tragedy', 'anyone', 'wants', 'go', 'pay', 'tribute', 'memory', 'suffered', 'killed'], tags=[0]),
 TaggedDocument(words=['centre', 'worker', 'chang', 'chi', 'chung', 'told', 'new', 'china', 'panda', 'breeding', 'risk', 'inbreeding'], tags=[1]),
 TaggedDocument(words=['however', 'veterinarians', 'taking', 'samples', 'eggs', 'chicken', 'meat', 'belgium', 'netherlands', 'france', 'italy', 'germany', 'poland', 'countries', 'products', 'problematic', 'insecticides', 'found', 'market'], tags=[2]),
 TaggedDocument(words=['user', 'handle', 'theepharoah', 'constantly', 'retweeted', 'onto', 'timeline'], tags=[3]),
 TaggedDocument(words=['time', 'lucie', 'years', 'old', 'definitely', 'want', 'put', 'brakes', 'career'], tags=[4]),
 TaggedDocument(words=['fighter'], tags=[5]),
 TaggedDocument(words=['three', 'weeks', 'later', 'jokowi', 'gave', 'impassioned', 'spee

In [11]:
## Train doc2vec model
model_Doc2Vec = Doc2Vec(tagged_data, vector_size = 1024, window = 3, min_count = 1.5, epochs = 100)



In [12]:
def Doc2Vec(sentece):
    x = model_Doc2Vec.infer_vector(sentece)    
    return x
    

In [13]:
#embedding = df_cp.apply(lambda x:Doc2Vec(x['w_reference']), axis=1)

In [14]:
#pd.DataFrame(embedding).to_csv('embedding.csv')

## Similarity 

## Doc2Vec + Cosine Similarity 

In [15]:
from scipy import spatial
def cosinesimilarity(ref,trans):
    t = model_Doc2Vec.infer_vector(trans)
    r = model_Doc2Vec.infer_vector(ref)
    result = 1 - spatial.distance.cosine(t,r)
    return result

REF: https://www.machinelearningplus.com/nlp/cosine-similarity/

The cosine similarity is advantageous because even if the two similar documents are far apart by the Euclidean distance (due to the size of the document), chances are they may still be oriented closer together. The smaller the angle, higher the cosine similarity.

**Interpretation** : A cosine value of 0 means that the two vectors are at 90 degrees to each other (orthogonal) and have no match. The closer the cosine value to 1, the smaller the angle and the greater the match between vectors. 

In [ ]:
df_cp['metric'] = df_cp.apply(lambda x:cosinesimilarity(x['w_reference'],x['w_translation']), axis=1)

In [ ]:
df_cp

In [ ]:
df_final = df_cp.drop(columns=['p_reference', 'p_translation','w_reference','w_translation'])

In [ ]:
df_final